In [ ]:
import pandas as pd
import sqlite3


In [ ]:
orders = pd.read_csv("orders.csv")
orders.head()



,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [ ]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [ ]:
# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Read SQL file
with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [ ]:
# Merge orders with users
df = orders.merge(users, on="user_id", how="left")

# Merge with restaurants
df = df.merge(restaurants, on="restaurant_id", how="left")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [ ]:
gold_orders = df[df["membership"] == "Gold"]
gold_orders.shape[0]


4987

In [ ]:
round(df[df["city"] == "Hyderabad"]["total_amount"].sum())


1889367

In [ ]:
df["user_id"].nunique()


2883

In [ ]:
round(gold_orders["total_amount"].mean(), 2)


np.float64(797.15)

In [ ]:
df[df["rating"] >= 4.5].shape[0]


3374

In [ ]:
gold_city_revenue = (
    gold_orders
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [ ]:
top_city = gold_city_revenue.idxmax()
gold_orders[gold_orders["city"] == top_city].shape[0]


1337

In [ ]:
top_city


'Chennai'

In [ ]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [ ]:
user_spend = df.groupby("user_id")["total_amount"].sum()
(user_spend > 1000).sum()


np.int64(2544)

In [ ]:
# Ensure rating is numeric
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")

# Remove rows with missing ratings
rating_df = df.dropna(subset=["rating"])

# Create rating buckets safely
rating_df["rating_range"] = pd.cut(
    rating_df["rating"],
    bins=[0, 3.5, 4.0, 4.5, 5.1],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"],
    include_lowest=True
)

# Revenue by rating range
rating_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19476\3962995117.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [ ]:
df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range', 'name_clean'],
      dtype='object')

In [ ]:
df.groupby(["membership", "cuisine"])["total_amount"] \
  .sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [ ]:
gold_orders = df[df["membership"] == "Gold"]

round((len(gold_orders) / len(df)) * 100)


50

In [ ]:
# Convert order_date safely
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True, errors="coerce")

# Drop invalid dates
date_df = df.dropna(subset=["order_date"])

# Create quarter column
date_df["quarter"] = date_df["order_date"].dt.quarter

# Revenue by quarter
date_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [ ]:
date_df["quarter_label"] = "Q" + date_df["quarter"].astype(str)

date_df.groupby("quarter_label")["total_amount"].sum().sort_values(ascending=False)


quarter_label
Q3    2037385.10
Q4    2018263.66
Q1    2010626.64
Q2    1945348.72
Name: total_amount, dtype: float64